In [1]:
import cudf as pd

In [2]:
dfUser = pd.read_csv("users.csv", sep='\t')
dfRel = pd.read_csv("relations.csv", sep='\t')

In [3]:
empty_nfriends_ids = dfUser.loc[dfUser['nFriends'].isnull(), 'id']

for user_id in empty_nfriends_ids.to_arrow().to_pylist():
    count = dfRel[dfRel['userID'] == user_id].shape[0]
    dfUser.loc[dfUser['id'] == user_id, 'nFriends'] = count

dfUser['nFriends'] = dfUser['nFriends'].fillna(0)

In [4]:
# loc函数取targetUser==1的用户的id
targetUsers = dfUser.loc[dfUser['targetUser'] == 1, ['id']]
#targetUsers = dfUser['id']

In [5]:
df1 = pd.merge(targetUsers, dfRel, left_on=['id'], right_on=['userID'])
df2 = pd.merge(df1, dfRel, left_on=['friendID'], right_on=['userID'])
df3 = df2.groupby(['id','friendID_y'])['friendID_x'].count().reset_index()
df3.rename(columns={'id':'userID','friendID_y':'ffID','friendID_x':'cn'}, inplace=True)
df3 = df3[df3['userID'] != df3['ffID']]

In [6]:
df4 = pd.merge(df3, dfUser, left_on='userID', right_on='id')
df5 = pd.merge(df4, dfUser, left_on='ffID', right_on='id')
df6 = pd.merge(df5, dfRel, how='left', left_on=['userID','ffID'], right_on=['userID','friendID'])
df6.loc[df6['friendID'].isnull(), ['friendID']]=0
df6.loc[df6['friendID']>0, ['friendID']]=1
df6.rename(columns={'friendID':'isFriend'}, inplace=True)
df6['isFriend']=df6['isFriend'].astype(int)

In [7]:
df6

,userID,ffID,cn,id_x,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,...,nLike_y,nPosts_y,nShares_y,nAlbums_y,nShortPosts_y,nVisits_y,location_y,privacy_y,timeline_y,isFriend
0,565648357,559044931,1,565648357,1,田,0.0,1.0,0.0,湖北,...,13.0,117.0,<NA>,8.0,273.0,1301.0,南京航空航天大学,1.0,0,0
1,587103545,587174273,1,587103545,1,于,1.0,1.0,0.0,山东,...,15.0,0.0,<NA>,7.0,126.0,1980.0,西安市,1.0,0,0
2,588859329,616474115,1,588859329,1,陈,<NA>,1.0,0.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,297.0,<NA>,1.0,1,0
3,685155163,146108507,1,685155163,1,吴,1.0,1.0,0.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,274.0,<NA>,0.0,0,0
4,553546433,755041001,5,553546433,1,杨,0.0,1.0,0.0,天津,...,1.0,1.0,<NA>,2.0,8.0,123.0,南昌市,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205437,632327215,621533225,2,632327215,1,李,0.0,1.0,1.0,<NA>,...,13.0,37.0,16.0,<NA>,<NA>,1468.0,<NA>,0.0,1,0
1205438,533661155,583971285,1,533661155,1,梁,0.0,1.0,0.0,黑龙江,...,<NA>,0.0,<NA>,0.0,0.0,17.0,槐林中学,1.0,0,0
1205439,584176205,572392025,4,584176205,1,李,0.0,1.0,0.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,5198.0,<NA>,1.0,1,0
1205440,550534377,764121423,1,550534377,1,陈,0.0,1.0,0.0,四川,...,14.0,0.0,<NA>,2.0,35.0,527.0,北京师范大学珠海分校,1.0,0,0


In [8]:
import cudf

columns_to_process = ['fName_y', 'gender_y', 'homeCity_y', 'location_y', 'privacy_y', 'homeProvince_y']

for column in columns_to_process:
    group_counts = df6.groupby(['userID', column], dropna=False).size().reset_index(name='counts')
    total_counts = df6.groupby(['userID'], dropna=False).agg({'nFriends_x': 'first'}).reset_index()
    
    friend_counts = df6[df6['isFriend'] == 1].groupby(['userID', column], dropna=False).size().reset_index(name='friend_counts')
    merged = group_counts.merge(total_counts, on='userID', how='left')
    merged = merged.merge(friend_counts, on=['userID', column], how='left', suffixes=('', '_friend'))
    
    merged['friend_counts'] = merged['friend_counts'].fillna(0)
    merged[f'per_{column}'] = merged['friend_counts'] / merged['nFriends_x']
    
    if f'per_{column}' in df6.columns:
        df6.drop(columns=[f'per_{column}'], inplace=True)
    df6 = df6.merge(merged[['userID', column, f'per_{column}']], on=['userID', column], how='left')


In [9]:
per_columns = [col for col in df6.columns if col.startswith('per_')]
#df6['per2'] = 1 - ((1 - df6['per_location_y'])*(1 - df6['per_fName_y'])*(1 - df6['per_gender_y'])*(1 - df6['per_homeCity_y'])*(1 - df6['per_privacy_y'])*(1 - df6['per_homeProvince_y']))
#df6['per1'] = 1 - (1 - df6['per_location_y'])
df10 = df6[['userID','ffID', 'isFriend','cn', 'nFriends_x', 'nFriends_y','per_fName_y', 'per_gender_y', 'per_homeCity_y', 'per_location_y', 'per_privacy_y', 'per_homeProvince_y']]

In [12]:
df10

,userID,ffID,isFriend,cn,nFriends_x,nFriends_y,per_fName_y,per_gender_y,per_homeCity_y,per_location_y,per_privacy_y,per_homeProvince_y
0,428836397,488076189,0,1,1079.0,377.0,0.012975,0.054680,0.139018,0.122335,0.316033,0.138091
1,531185749,615326487,0,1,1125.0,1569.0,0.024000,0.207111,0.109333,0.000000,0.295111,0.117333
2,156984681,613305177,0,3,913.0,475.0,0.003286,0.044907,0.109529,0.085433,0.351588,0.107338
3,544081007,545390805,0,1,393.0,2119.0,0.010178,0.160305,0.109415,0.083969,0.053435,0.104326
4,728351165,151198589,0,3,163.0,307.0,0.000000,0.042945,0.116564,0.067485,0.368098,0.110429
...,...,...,...,...,...,...,...,...,...,...,...,...
1205437,356577045,587062537,0,1,461.0,1321.0,0.002169,0.052061,0.110629,0.093275,0.036876,0.110629
1205438,609046557,566485275,0,2,429.0,941.0,0.004662,0.046620,0.200466,0.114219,0.314685,0.195804
1205439,607879043,624450051,0,1,831.0,43.0,0.008424,0.066185,0.200963,0.125150,0.008424,0.197353
1205440,139840069,598024527,0,1,337.0,471.0,0.000000,0.219585,0.005935,0.000000,0.326409,0.008902


In [13]:
import cudf
is_friend_counts = df10[df10['isFriend'] == 1].groupby('userID').size()
filtered_user_ids = is_friend_counts[(is_friend_counts >= 100) & (is_friend_counts <= 200000)]
print(f'UserIDs with isFriend=1 count between 0 and 10:\n{filtered_user_ids}')


UserIDs with isFriend=1 count between 0 and 10:
userID
601308405    169
566404705    242
617151123    255
724535251    188
543051191    430
            ... 
588385359    105
589446243    646
547030075    140
174893247    260
532053793    226
Length: 158, dtype: int64


In [9]:
import cudf
import numpy as np

friend_counts = df10[df10['isFriend'] == 1].groupby('userID').size()
selected_user_ids = friend_counts[friend_counts >= 100].index

df10['change'] = 0

selected_user_ids_np = selected_user_ids.values_host

for user_id in selected_user_ids_np:
    user_rows = df10[(df10['userID'] == user_id) & (df10['isFriend'] == 1)]
    change_count = np.floor(len(user_rows) * 0.10).astype(int)
    if change_count > 0:
        # 使用 user_rows.index.values_host 以获得 NumPy 数组
        indices_to_change = np.random.choice(user_rows.index.values_host, change_count, replace=False)
        df10.loc[indices_to_change, 'isFriend'] = 0
        df10.loc[indices_to_change, 'change'] = 1


In [13]:
if isinstance(selected_user_ids, (cudf.Series, cudf.Index)):
    selected_user_ids_list = selected_user_ids.to_pandas().tolist()
else:
    selected_user_ids_list = selected_user_ids

filtered_df10 = df10[df10['userID'].isin(selected_user_ids_list)]

In [13]:
#filtered_df10.to_csv("cudf_change.csv", sep='\t', index=False)
import cudf as pd
df10 = pd.read_csv("cudf_change.csv", sep='\t')

In [14]:
def standardize_data(df, columns):
    for col in columns:
        col_mean = df[col].mean()
        col_std = df[col].std()
        df[col] = (df[col] - col_mean) / col_std
    return df

In [15]:
columns = ['nFriends_x', 'nFriends_y', 'cn', 'per_fName_y','per_gender_y', 'per_homeCity_y', 'per_location_y', 'per_privacy_y','per_homeProvince_y']
df10_standardized = standardize_data(df10.copy(), columns)

In [16]:
df10_standardized.to_csv("cudf_change1.csv", sep='\t', index=False)
import cudf as pd
#df10_standardized = pd.read_csv("cudf_change1.csv", sep='\t')

In [21]:
def calculate1_factor_values(df, factor_expression):
    return df.eval(factor_expression)

def nnget_average_pAT10_for_user(df, factor_expression, user_id):
    df['factor_value'] = calculate1_factor_values(df, factor_expression)
    user_df = df[(df['userID'] == user_id) & (df['isFriend'] == 0)]
    top10_per_user = user_df.sort_values(['userID', 'factor_value'], ascending=[True, False]).head(10)
    relevant_count = top10_per_user[top10_per_user['change'] == 1].shape[0]
    pAT10 = relevant_count / 10.0
    del df['factor_value']
    return pAT10

user_id = 205820795
average_pAT10 = nnget_average_pAT10_for_user(df10_standardized, "0.602015 * cn + 0.338696 * per_homeProvince_y + 0.260863 * per_homeProvince_y + 0.8368 * per_fName_y + 0.369644 * nFriends_y % 0.4847 * cn * per_gender_y + 0.693617 * cn", user_id)
print(average_pAT10)

0.5


In [36]:
import cudf
df10_standardized = pd.read_csv("cudf.csv", sep='\t')
df10_truth= pd.read_csv("cudf_tru20_al.csv", sep='\t')

def get_pAT(df, factor_expression, weight_pAT5=0.7, weight_pAT10=0.3):
    df['factor_value'] = df.eval(factor_expression)
    sorted_df = df.sort_values(['userID', 'factor_value'], ascending=[True, False])
    top10_per_user = sorted_df.groupby('userID').head(10)
    top10_per_user['relevant'] = top10_per_user['isFriend']
    user_relevant_counts = top10_per_user.groupby('userID')['relevant'].sum().reset_index(name='relevant_count')
    user_relevant_counts['pAT5'] = user_relevant_counts['relevant_count'].clip(upper=5) / 5.0
    user_relevant_counts['pAT10'] = user_relevant_counts['relevant_count'] / 10.0
    average_pAT = (user_relevant_counts['pAT5'] * weight_pAT5 + user_relevant_counts['pAT10'] * weight_pAT10).mean()
    return average_pAT


def calculate1_factor_values(df, factor_expression):
    return df.eval(factor_expression)

def nnget_average_pAT10(df, factor_expression):
    df['factor_value'] = df.eval(factor_expression)
    isFriend_zero_df = df[df['isFriend'] == 0]
    top10_per_user = isFriend_zero_df.sort_values(['userID', 'factor_value'], ascending=[True, False]).groupby('userID').head(10)
    pAT10 = (top10_per_user['change'] == 1).groupby(top10_per_user['userID']).sum() / 10.0
    all_users_pAT10 = pAT10.reindex(df['userID'].unique(), fill_value=0)
    avg_pAT10 = all_users_pAT10.mean()
    del df['factor_value']
    return avg_pAT10

def nnget_average_pAT10_and_user_scores(df, factor_expression):
    df['factor_value'] = calculate1_factor_values(df, factor_expression)
    isFriend_zero_df = df[df['isFriend'] == 0]
    top10_per_user = isFriend_zero_df.sort_values(['userID', 'factor_value'], ascending=[True, False]).groupby('userID').head(10)
    relevant_count = top10_per_user[top10_per_user['change'] == 1].groupby('userID').size()
    pAT10 = relevant_count.reindex(isFriend_zero_df['userID'].unique(), fill_value=0) / 10.0
    avg_pAT10 = pAT10.mean()
    user_pAT10_df = pAT10.reset_index().rename(columns={0: 'pAT10'})
    return avg_pAT10, user_pAT10_df

factor_expr = "0.602015 * cn + 0.338696 * per_homeProvince_y + 0.260863 * per_homeProvince_y + 0.8368 * per_fName_y + 0.369644 * nFriends_y % 0.4847 * cn * per_gender_y + 0.693617 * cn"
avg_pAT10 = get_pAT(df10_standardized, factor_expr)
print(avg_pAT10)

0.8069421487603305


In [4]:
import cudf as pd
df10_standardized = pd.read_csv("cudf.csv", sep='\t')
df10_truth= pd.read_csv("cudf_tru20_al.csv", sep='\t')

def get_pAT(df, factor_expression, weight_pAT5=0.7, weight_pAT10=0.3):
    df['factor_value'] = df.eval(factor_expression)
    sorted_df = df.sort_values(['userID', 'factor_value'], ascending=[True, False])
    top10_per_user = sorted_df.groupby('userID').head(10)
    top10_per_user['relevant'] = top10_per_user['isFriend']
    user_relevant_counts = top10_per_user.groupby('userID')['relevant'].sum().reset_index(name='relevant_count')
    user_relevant_counts['pAT5'] = user_relevant_counts['relevant_count'].clip(upper=5) / 5.0
    user_relevant_counts['pAT10'] = user_relevant_counts['relevant_count'] / 10.0
    average_pAT = (user_relevant_counts['pAT5'] * weight_pAT5 + user_relevant_counts['pAT10'] * weight_pAT10).mean()
    return average_pAT

def nnget_average_pAT10(df, factor_expression):
    df['factor_value'] = df.eval(factor_expression)
    isFriend_zero_df = df[df['isFriend'] == 0]
    top10_per_user = isFriend_zero_df.sort_values(['userID', 'factor_value'], ascending=[True, False]).groupby('userID').head(10)
    pAT5 = (top10_per_user.groupby('userID').head(5)['change'] == 1).groupby(top10_per_user['userID']).sum() / 5
    all_users_pAT5 = pAT5.reindex(df['userID'].unique(), fill_value=0)
    avg_pAT5 = all_users_pAT5.mean()
    pAT10 = (top10_per_user['change'] == 1).groupby(top10_per_user['userID']).sum() / 10
    all_users_pAT10 = pAT10.reindex(df['userID'].unique(), fill_value=0)
    avg_pAT10 = all_users_pAT10.mean()
    avg_pAT = (0.7 * avg_pAT5 + 0.3 * avg_pAT10) * 0.95
    return avg_pAT

factor_expr = "0.119 * cn % 0.594 * per_privacy_y + per_privacy_y * cn + per_location_y % 0.7164 * cn + 0.0557 % 0.4554 * per_homeProvince_y + per_homeProvince_y // nFriends_y % 0.035799 / per_homeProvince_y * per_homeCity_y - 0.1409 * cn * per_privacy_y // 0.4068 * nFriends_y + 0.1165 * per_gender_y - nFriends_y + 0.454168 + 0.0358 % cn + nFriends_y % 0.825845 * cn"
avg_pAT1 = get_pAT(df10_standardized, factor_expr)
print(avg_pAT1)
avg_pAT2 = nnget_average_pAT10(df10_truth, factor_expr)
print(avg_pAT2)

print((0.7 * avg_pAT1) + (0.3 * avg_pAT2))

0.8165289256198347
0.8479999999999999
0.8259702479338842


In [ ]:
import random
import cudf as pd
import cupy as np
df10_standardized = pd.read_csv("cudf.csv", sep='\t')
df10_truth= pd.read_csv("cudf_tru20_al.csv", sep='\t')

def get_pAT(df, factor_expression, weight_pAT5=0.7, weight_pAT10=0.3):
    df['factor_value'] = df.eval(factor_expression)
    sorted_df = df.sort_values(['userID', 'factor_value'], ascending=[True, False])
    top10_per_user = sorted_df.groupby('userID').head(10)
    top10_per_user['relevant'] = top10_per_user['isFriend']
    user_relevant_counts = top10_per_user.groupby('userID')['relevant'].sum().reset_index(name='relevant_count')
    user_relevant_counts['pAT5'] = user_relevant_counts['relevant_count'].clip(upper=5) / 5.0
    user_relevant_counts['pAT10'] = user_relevant_counts['relevant_count'] / 10.0
    average_pAT = (user_relevant_counts['pAT5'] * weight_pAT5 + user_relevant_counts['pAT10'] * weight_pAT10).mean()
    return average_pAT

def nnget_average_pAT10(df, factor_expression):
    df['factor_value'] = df.eval(factor_expression)
    isFriend_zero_df = df[df['isFriend'] == 0]
    top10_per_user = isFriend_zero_df.sort_values(['userID', 'factor_value'], ascending=[True, False]).groupby('userID').head(10)
    pAT5 = (top10_per_user.groupby('userID').head(5)['change'] == 1).groupby(top10_per_user['userID']).sum() / 5
    all_users_pAT5 = pAT5.reindex(df['userID'].unique(), fill_value=0)
    avg_pAT5 = all_users_pAT5.mean()
    pAT10 = (top10_per_user['change'] == 1).groupby(top10_per_user['userID']).sum() / 10
    all_users_pAT10 = pAT10.reindex(df['userID'].unique(), fill_value=0)
    avg_pAT10 = all_users_pAT10.mean()
    return avg_pAT5, avg_pAT10

def fitness_function(df_standardized, df_truth, factor_expression):
    avg_pAT1 = get_pAT(df_standardized, factor_expression)
    avg_pAT5, avg_pAT10 = nnget_average_pAT10(df_truth, factor_expression)
    avg_pAT2 = ((0.7 * avg_pAT5) + (0.3 * avg_pAT10))*0.95
    return (0.7 * avg_pAT1) + (0.3 * avg_pAT2)

def generate_initial_population(size, columns):
    population = []
    operators = ['+', '-', '*', '/','%','//','**']
    for _ in range(size):
        const_factor = round(random.uniform(0, 1), 6)
        expr = f"{const_factor} * {random.choice(columns)}"
        for _ in range(random.randint(1, 7)):  # Generate expressions with depth between 1 and 5
            const_factor = round(random.uniform(0, 1), 6)
            expr += f" {random.choice(operators)} {const_factor} * {random.choice(columns)}"
        population.append(expr)
    return population


def select(population, fitnesses, num_parents):
    parents = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)[:num_parents]
    return [expr for expr, _ in parents]

def crossover(parent1, parent2):
    parts1 = parent1.split(' ')
    parts2 = parent2.split(' ')
    if len(parts1) > 1 and len(parts2) > 1:
        crossover_point = random.randint(1, min(len(parts1), len(parts2)) - 1)
        child = ' '.join(parts1[:crossover_point] + parts2[crossover_point:])
        return child
    return parent1

def mutate(expression, columns, mutation_rate=0.1):
    parts = expression.split(' ')
    if random.random() < mutation_rate:
        index = random.randint(0, len(parts) - 1)
        if index % 2 == 0:
            const_factor = round(random.uniform(0, 1), 4)
            parts[index] = f"{const_factor} * {random.choice(columns)}"
        else:
            parts[index] = random.choice(['+', '-', '*', '/','%','//','**'])
    return ' '.join(parts)


def genetic_algorithm(df_standardized, df_truth, columns, population_size, num_generations, num_parents, mutation_rate):
    population = generate_initial_population(population_size, columns)
    for generation in range(num_generations):
        fitnesses = [fitness_function(df_standardized, df_truth, expr) for expr in population]
        max_index = fitnesses.index(max(fitnesses))
        best_expr = population[max_index]
        best_fitness = fitnesses[max_index]
        print(f'Generation {generation + 1}: Best fitness {best_fitness}, Best expression {best_expr}')

        parents = select(population, fitnesses, num_parents)
        next_population = parents[:2]  # Keep the best individuals

        while len(next_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            child = mutate(child, columns, mutation_rate)
            next_population.append(child)
        population = next_population
    best_expr = select(population, [fitness_function(df_standardized, df_truth, expr) for expr in population], 1)[0]
    return best_expr

columns = ['nFriends_x', 'nFriends_y', 'cn', 'per_fName_y', 'per_gender_y', 'per_homeCity_y', 'per_location_y', 'per_privacy_y', 'per_homeProvince_y']
best_expression = genetic_algorithm(df10_standardized, df10_truth, columns, population_size=5200, num_generations=100, num_parents=800, mutation_rate=0.3)
print('Best Expression:', best_expression)

Generation 152: Best fitness 0.8107438016528926, Best expression 0.0004 * per_location_y + per_homeCity_y % 0.052013 % cn + 0.181411 * 0.155747 * 0.155747 % per_fName_y // 0.155747 % per_fName_y + 0.3758 * per_homeCity_y * per_homeCity_y % 0.252208 * 0.9895 * cn